# 📊 Sales Data Exploration - Your First Pandas Adventure!

Welcome to your comprehensive sales data analysis project! This notebook will guide you through exploring e-commerce sales data using pandas.

## 🎯 What You'll Learn
- Loading and inspecting datasets
- Data quality assessment
- Basic statistical analysis
- Data relationships and patterns
- Essential pandas operations

## 📋 Prerequisites
- Basic Python knowledge
- Pandas fundamentals (Series, DataFrame, indexing)
- Understanding of data types

Let's start exploring! 🚀

## 1. 📚 Import Required Libraries

First, let's import all the libraries we'll need for our data exploration journey.

In [ ]:
# Core libraries for data analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For interactive widgets
import ipywidgets as widgets
from IPython.display import display

# System libraries
import os
import warnings
from datetime import datetime, timedelta

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

## 2. 📁 Load Sales Data

Now let's load our datasets! We have four related tables:
- **Customers**: Customer information and demographics
- **Products**: Product catalog with prices and categories  
- **Orders**: Order details and shipping information
- **Sales**: Individual sale transactions with quantities and prices

### 🔧 Practice Tip
Before running this cell, make sure you've generated the sample data by running:
```bash
python data/sample_data_generator.py
```

In [ ]:
# Define data file paths
data_path = '../data/raw/'

# Load all datasets
print("🔄 Loading datasets...")

try:
    # Load each dataset
    customers_df = pd.read_csv(f'{data_path}customers.csv')
    products_df = pd.read_csv(f'{data_path}products.csv')
    orders_df = pd.read_csv(f'{data_path}orders.csv')
    sales_df = pd.read_csv(f'{data_path}sales.csv')
    
    print("✅ All datasets loaded successfully!")
    
    # Display dataset sizes
    print(f"\n📊 Dataset Sizes:")
    print(f"├── Customers: {len(customers_df):,} records")
    print(f"├── Products: {len(products_df):,} records") 
    print(f"├── Orders: {len(orders_df):,} records")
    print(f"└── Sales: {len(sales_df):,} records")
    
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("💡 Make sure to run 'python data/sample_data_generator.py' first!")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## 3. 🔍 Data Inspection & Quality Assessment

Let's get familiar with our data! We'll examine the structure, data types, and quality of each dataset.

### 🎯 Key pandas methods you'll practice:
- `.head()` and `.tail()` - Quick data preview
- `.info()` - Data types and memory usage
- `.describe()` - Statistical summaries
- `.shape` - Dataset dimensions
- `.columns` - Column names
- `.dtypes` - Data types
- `.isnull()` - Missing value detection

In [ ]:
# 👥 CUSTOMERS DATASET EXPLORATION
print("="*60)
print("👥 CUSTOMERS DATASET")
print("="*60)

print("📋 Basic Info:")
print(f"Shape: {customers_df.shape}")
print(f"Columns: {list(customers_df.columns)}")

print("\n🔍 First 3 records:")
display(customers_df.head(3))

print("\n📊 Data Types & Missing Values:")
display(customers_df.info())

print("\n📈 Statistical Summary (numerical columns):")
display(customers_df.describe())

print("\n🔎 Missing Values Check:")
missing_values = customers_df.isnull().sum()
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "✅ No missing values found!")

print("\n🎯 Customer Segments Distribution:")
print(customers_df['customer_segment'].value_counts())

print("\n🎂 Age Group Distribution:")
print(customers_df['age_group'].value_counts())

In [ ]:
# 🛍️ PRODUCTS DATASET EXPLORATION
print("="*60)
print("🛍️ PRODUCTS DATASET")
print("="*60)

print("📋 Basic Info:")
print(f"Shape: {products_df.shape}")
print(f"Columns: {list(products_df.columns)}")

print("\n🔍 First 3 records:")
display(products_df.head(3))

print("\n📊 Data Types:")
print(products_df.dtypes)

print("\n📈 Price Statistics:")
display(products_df[['price', 'cost', 'weight_kg', 'rating']].describe())

print("\n🏷️ Categories Distribution:")
print(products_df['category'].value_counts())

print("\n🎯 Price Ranges:")
products_df['price_range'] = pd.cut(products_df['price'], 
                                   bins=[0, 50, 100, 300, 1000, 3000], 
                                   labels=['$0-50', '$50-100', '$100-300', '$300-1000', '$1000+'])
print(products_df['price_range'].value_counts())

print("\n⭐ Rating Distribution:")
print(products_df['rating'].value_counts().sort_index())

## 4. 📅 Data Cleaning and Preprocessing

Time to clean our data and prepare it for analysis! This is where pandas really shines.

### 🎯 What we'll do:
- Convert date columns to datetime format
- Handle any data type issues
- Create derived columns for analysis
- Merge datasets for comprehensive analysis

In [ ]:
# 🧹 DATA CLEANING & PREPROCESSING

print("🧹 Cleaning and preprocessing data...")

# 1. Convert date columns to datetime
print("📅 Converting date columns...")
customers_df['registration_date'] = pd.to_datetime(customers_df['registration_date'])
products_df['launch_date'] = pd.to_datetime(products_df['launch_date'])
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
orders_df['shipped_date'] = pd.to_datetime(orders_df['shipped_date'])
orders_df['delivered_date'] = pd.to_datetime(orders_df['delivered_date'])

# 2. Create a comprehensive sales dataset by merging tables
print("🔗 Creating comprehensive sales dataset...")
sales_analysis_df = sales_df.merge(orders_df, on='order_id', how='left') \
                           .merge(products_df, on='product_id', how='left') \
                           .merge(customers_df, on='customer_id', how='left')

print(f"✅ Created comprehensive dataset with {len(sales_analysis_df):,} records")

# 3. Add derived columns for analysis
print("➕ Adding derived columns...")

# Calculate profit margin
sales_analysis_df['profit_margin'] = (sales_analysis_df['profit'] / sales_analysis_df['total_amount']) * 100

# Extract time-based features
sales_analysis_df['year'] = sales_analysis_df['order_date'].dt.year
sales_analysis_df['month'] = sales_analysis_df['order_date'].dt.month
sales_analysis_df['quarter'] = sales_analysis_df['order_date'].dt.quarter
sales_analysis_df['day_of_week'] = sales_analysis_df['order_date'].dt.day_name()
sales_analysis_df['month_name'] = sales_analysis_df['order_date'].dt.month_name()

# Calculate delivery time
sales_analysis_df['delivery_days'] = (sales_analysis_df['delivered_date'] - sales_analysis_df['order_date']).dt.days

print("✅ Data preprocessing complete!")

# Display the enhanced dataset
print(f"\n📊 Enhanced Sales Dataset:")
print(f"Shape: {sales_analysis_df.shape}")
print(f"Date range: {sales_analysis_df['order_date'].min()} to {sales_analysis_df['order_date'].max()}")

display(sales_analysis_df.head(3))

## 5. 📈 Summary Statistics & Key Metrics

Now let's calculate some key business metrics using pandas aggregation functions!

### 🎯 Pandas skills you'll practice:
- `.sum()`, `.mean()`, `.count()` - Basic aggregations
- `.groupby()` - Group-wise operations  
- `.agg()` - Multiple aggregations
- `.value_counts()` - Frequency analysis
- `.quantile()` - Percentile calculations

In [ ]:
# 📊 KEY BUSINESS METRICS

print("="*60)
print("💰 OVERALL BUSINESS PERFORMANCE")
print("="*60)

# Overall metrics
total_revenue = sales_analysis_df['total_amount'].sum()
total_profit = sales_analysis_df['profit'].sum()
total_orders = sales_analysis_df['order_id'].nunique()
total_customers = sales_analysis_df['customer_id'].nunique()
total_products_sold = sales_analysis_df['quantity'].sum()
avg_order_value = sales_analysis_df.groupby('order_id')['total_amount'].sum().mean()

print(f"💵 Total Revenue: ${total_revenue:,.2f}")
print(f"💰 Total Profit: ${total_profit:,.2f}")
print(f"📈 Profit Margin: {(total_profit/total_revenue)*100:.1f}%")
print(f"🛒 Total Orders: {total_orders:,}")
print(f"👥 Unique Customers: {total_customers:,}")
print(f"📦 Total Products Sold: {total_products_sold:,}")
print(f"💳 Average Order Value: ${avg_order_value:.2f}")

print(f"\n" + "="*60)
print("📅 TIME-BASED ANALYSIS")
print("="*60)

# Monthly performance
monthly_sales = sales_analysis_df.groupby(['year', 'month']).agg({
    'total_amount': 'sum',
    'profit': 'sum',
    'order_id': 'nunique',
    'quantity': 'sum'
}).round(2)

print("📊 Monthly Sales Summary (Last 6 months):")
display(monthly_sales.tail(6))

# Best performing month
best_month = sales_analysis_df.groupby('month_name')['total_amount'].sum().sort_values(ascending=False)
print(f"\n🏆 Best performing month: {best_month.index[0]} (${best_month.iloc[0]:,.2f})")

print(f"\n" + "="*60)
print("🏷️ PRODUCT PERFORMANCE")
print("="*60)

# Category performance
category_performance = sales_analysis_df.groupby('category').agg({
    'total_amount': 'sum',
    'profit': 'sum',
    'quantity': 'sum',
    'order_id': 'nunique'
}).round(2).sort_values('total_amount', ascending=False)

print("🛍️ Sales by Category:")
display(category_performance)

# Top 10 products by revenue
top_products = sales_analysis_df.groupby(['product_id', 'product_name']).agg({
    'total_amount': 'sum',
    'quantity': 'sum'
}).round(2).sort_values('total_amount', ascending=False).head(10)

print("\n🌟 Top 10 Products by Revenue:")
display(top_products)

## 6. 📊 Sales Trend Visualization

Time to visualize our data! We'll create compelling charts to understand sales patterns and trends.

### 🎯 Visualization skills you'll practice:
- Line plots for time series trends
- Bar charts for categorical comparisons
- Heatmaps for correlation analysis
- Multiple subplots for comprehensive views

In [ ]:
# 📈 SALES TREND VISUALIZATIONS

# Set up the plotting area
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Sales Performance Dashboard', fontsize=16, fontweight='bold')

# 1. Monthly Revenue Trend
monthly_revenue = sales_analysis_df.groupby(sales_analysis_df['order_date'].dt.to_period('M'))['total_amount'].sum()
monthly_revenue.plot(kind='line', ax=axes[0,0], marker='o', linewidth=2, markersize=6)
axes[0,0].set_title('📈 Monthly Revenue Trend')
axes[0,0].set_ylabel('Revenue ($)')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Daily Sales Pattern (by day of week)
daily_pattern = sales_analysis_df.groupby('day_of_week')['total_amount'].sum().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
daily_pattern.plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('📅 Sales by Day of Week')
axes[0,1].set_ylabel('Revenue ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Category Performance
category_sales = sales_analysis_df.groupby('category')['total_amount'].sum().sort_values(ascending=True)
category_sales.plot(kind='barh', ax=axes[1,0], color='skyblue')
axes[1,0].set_title('🏷️ Revenue by Category')
axes[1,0].set_xlabel('Revenue ($)')

# 4. Customer Segment Analysis
segment_sales = sales_analysis_df.groupby('customer_segment')['total_amount'].sum()
colors = ['gold', 'lightgreen', 'lightblue']
axes[1,1].pie(segment_sales.values, labels=segment_sales.index, autopct='%1.1f%%', 
              colors=colors, startangle=90)
axes[1,1].set_title('👥 Revenue by Customer Segment')

plt.tight_layout()
plt.show()

# Additional interactive plot with Plotly
print("\n🎯 Interactive Monthly Trend (with Plotly):")
monthly_data = sales_analysis_df.groupby(sales_analysis_df['order_date'].dt.to_period('M')).agg({
    'total_amount': 'sum',
    'profit': 'sum',
    'order_id': 'nunique'
}).reset_index()

monthly_data['order_date'] = monthly_data['order_date'].astype(str)

fig_plotly = make_subplots(specs=[[{"secondary_y": True}]])
fig_plotly.add_trace(
    go.Scatter(x=monthly_data['order_date'], y=monthly_data['total_amount'], 
               name="Revenue", line=dict(color='blue', width=3)),
    secondary_y=False,
)
fig_plotly.add_trace(
    go.Scatter(x=monthly_data['order_date'], y=monthly_data['order_id'], 
               name="Orders", line=dict(color='red', width=3)),
    secondary_y=True,
)

fig_plotly.update_xaxes(title_text="Month")
fig_plotly.update_yaxis(title_text="Revenue ($)", secondary_y=False)
fig_plotly.update_yaxis(title_text="Number of Orders", secondary_y=True)
fig_plotly.update_layout(title_text="📊 Monthly Revenue & Orders Trend", height=400)

fig_plotly.show()

## 7. 🎯 Practice Exercises

Now it's your turn! Try these exercises to strengthen your pandas skills.

### 🔥 Challenge 1: Customer Analysis
Find the top 10 customers by total spending and analyze their purchasing patterns.

### 🔥 Challenge 2: Seasonal Trends  
Identify which quarters perform best and create a seasonal analysis.

### 🔥 Challenge 3: Product Insights
Calculate the profit margin by product category and find the most profitable products.

### 🔥 Challenge 4: Geographic Analysis
Analyze sales performance by state/region (use the customer location data).

### 🔥 Challenge 5: Time-based Patterns
Find the best-selling day of the week for each product category.

**💡 Tip**: Use the cells below to work on these challenges!

In [ ]:
# 🎯 YOUR PRACTICE SPACE
# Use this cell to work on the challenges above!

# Example: Challenge 1 - Top 10 customers by total spending
print("Challenge 1: Top 10 Customers by Total Spending")
print("="*50)

top_customers = sales_analysis_df.groupby(['customer_id', 'first_name', 'last_name']).agg({
    'total_amount': 'sum',
    'order_id': 'nunique',
    'quantity': 'sum'
}).round(2).sort_values('total_amount', ascending=False).head(10)

display(top_customers)

# Add your own analysis below:


## 🎉 Congratulations!

You've completed your first comprehensive pandas data exploration! Here's what you've learned:

### ✅ Skills Mastered:
- **Data Loading**: Reading CSV files with `pd.read_csv()`
- **Data Inspection**: Using `.head()`, `.info()`, `.describe()`, `.shape`
- **Data Cleaning**: Converting data types, handling dates
- **Data Merging**: Combining multiple datasets with `.merge()`
- **Aggregation**: Using `.groupby()`, `.sum()`, `.mean()`, `.count()`
- **Visualization**: Creating charts with matplotlib and plotly
- **Time Series**: Working with datetime data and extracting features

### 🚀 Next Steps:
1. **Advanced Analysis**: Move to `02_data_cleaning.ipynb` for deeper data preprocessing
2. **Customer Analytics**: Explore `04_customer_analysis.ipynb` for customer segmentation
3. **Dashboard Creation**: Build interactive dashboards in `05_dashboard_creation.ipynb`

### 💡 Keep Practicing:
- Try the challenge exercises above
- Experiment with different visualizations
- Ask business questions and use pandas to find answers!

**Happy Analyzing! 📊✨**